In [3]:
!pip install langchain langchain-community pypdf sentence_transformers openai tiktoken -q
!pip install faiss-cpu tokenizer unstructured -q
!pip install numpy pandas nltk -q

In [1]:
import sys, os
import torch
import textwrap
from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.embeddings import HuggingFaceEmbeddings

import numpy as np
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     /teamspace/studios/this_studio/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /teamspace/studios/this_studio/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
URLs=[
    'https://blog.gopenai.com/paper-review-llama-2-open-foundation-and-fine-tuned-chat-models-23e539522acb',
    'https://www.mosaicml.com/blog/mpt-7b',
    'https://stability.ai/blog/stability-ai-launches-the-first-of-its-stablelm-suite-of-language-models',
    'https://lmsys.org/blog/2023-03-30-vicuna/'
]

In [4]:
loader = UnstructuredURLLoader(URLs)
data = loader.load()

In [5]:
# data
len(data)

4

In [6]:
text_splitter = CharacterTextSplitter(separator='\n', chunk_size=1000, chunk_overlap=100)

# split the data
chunks_data = text_splitter.split_documents(data)

In [7]:
print(len(chunks_data))
print(chunks_data[0])

84
page_content='Open in app
Sign up
Sign in
Write
Sign up
Sign in
Paper Review
Paper Review: Llama 2: Open Foundation and Fine-Tuned Chat Models
Llama 2: one of the best open source models
Andrew Lukyanenko
Follow
Published in
GoPenAI
15 min read
Jul 20, 2023
--
Project link
Model link
Paper link
The authors of the work present Llama 2, an assortment of pretrained and fine-tuned large language models (LLMs) with sizes varying from 7 billion to 70 billion parameters. The fine-tuned versions, named Llama 2-Chat, are specifically designed for dialogue applications. These models surpass the performance of existing open-source chat models on most benchmarks, and according to human evaluations for usefulness and safety, they could potentially replace closed-source models. The authors also detail their approach to fine-tuning and safety enhancements for Llama 2-Chat to support the community in further developing and responsibly handling LLMs.
Pretraining' metadata={'source': 'https://blog.go

In [19]:
embeddings = HuggingFaceEmbeddings(model_name='facebook/bart-large-cnn')

No sentence-transformers model found with name facebook/bart-large-cnn. Creating a new one with mean pooling.


In [20]:
query_result = embeddings.embed_query("Hello world")
len(query_result)

1024

In [ ]:
# embeddings = OpenAIEmbeddings()
# print(embeddings)

# query_result = embeddings.embed_query("Hello world")
# len(query_result)

In [21]:
vector_db =FAISS.from_documents(chunks_data, embeddings)

### https://python.langchain.com/docs/integrations/chat/huggingface/

In [19]:
%pip install --upgrade --quiet  langchain-huggingface text-generation transformers google-search-results numexpr langchainhub sentencepiece jinja2 bitsandbytes accelerate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [22]:
import getpass
import os

os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass.getpass(
    "Enter your Hugging Face API key: "
)

In [ ]:
# llm = ChatOpenAI()

In [23]:
!pip install sentencepiece -q

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [25]:
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
)

In [26]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint

llm = HuggingFaceEndpoint(
    repo_id="facebook/bart-large-cnn",
    task="text-generation",
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.03,
    model_kwargs={"quantization_config": quantization_config},
)

chat_model = ChatHuggingFace(llm=llm)

In [30]:
from langchain_core.messages import (
    HumanMessage,
    SystemMessage,
)

messages = [
    SystemMessage(content="You're a helpful assistant"),
    HumanMessage(
        content="What happens when an unstoppable force meets an immovable object?"
    ),
]

# ai_msg = chat_model.invoke('give me a short summary of harry potter book')
ai_msg = chat_model.invoke('give me a short summary of harry potter book')

BadRequestError: (Request ID: ox0x3v7kAWetyXFcXeRUL)

Bad request:
`args[0]`: {'model': 'facebook/bart-large-cnn', 'messages': [{'role': 'user', 'content': 'give me a short summary of harry potter book'}], 'stream': False} have the wrong format. The should be either of type `str` or type `list`